In [1]:
import numpy as np

In [2]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

## 1. RNN lan truyền thuận

<img src="images/rnn_step_forward.png" style="width:700px;height:300px;">

In [3]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Thực hiện một RNN-cell 

    Arguments:
        xt -- dữ liệu đầu vào tại thời điểm "t", mảng numpy có dạng (n_x, m).
        a_prev -- Trạng thái ẩn tại thời điểm "t-1", mảng numpy có dạng (n_a, m)

    Parameters: Dictionary chứa:
        Wax -- Ma trận trọng số nhân với đầu vào, mảng numpy có dạng (n_a, n_x)
        Waa -- Ma trận trọng số nhân với trạng thái ẩn, mảng numpy có dạng (n_a, n_a)
        Wya -- Ma trận trọng số liên quan đến trạng thái ẩn và đầu ra, mảng numpy có dạng (n_y, n_a)

        ba -- Sai số, mảng numpy có dạng (n_a, 1)
        by -- Sai số, liên quan đến trạng thái ẩn và đầu ra, mảng numpy có dạng (n_y, 1)
    
    Returns:
        a_next -- trạng thái ẩn tiếp theo, có dạng (n_a, m)
        yt_pred -- dự đoán tại thời điểm "t", mảng numpy có dạng (n_y, m)
        cache -- tuple các giá trị cần thiết cho quá trình lan truyền ngược, chứa (a_next, a_prev, xt, parameters)
    """
   
    # Truy xuất các tham số từ "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
   
    # tính toán trạng thái kích hoạt tiếp theo sử dụng công thức được đưa ra trên
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    
    # tính toán đầu ra của cell hiện tại sử dụng công thức được đưa ra trên
    yt_pred = softmax(np.dot(Wya, a_next) + by)
   
    # lưu các giá trị cần thiết cho lan truyền ngược trong cache
    cache = (a_next, a_prev, xt, parameters)
   
    return a_next, yt_pred, cache


In [4]:
np.random.seed(1)

Wya = np.random.randn(2,5) # ma trận (2 hàng, 5 cột)
Waa = np.random.randn(5,5) # ma trận (5 hàng, 5 cột)
a_prev = np.random.randn(5,10) # ma trận (5 hàng, 10 cột)

Wax = np.random.randn(5,3) # ma trận (5 hàng, 3 cột)
xt = np.random.randn(3,10) # ma trận (3 hàng, 10 cột)

ba = np.random.randn(5,1) # ma trận (5 hàng, 1 cột)
by = np.random.randn(2,1) # ma trận (2 hàng, 1 cột)

# 3 trạng thái ẩn, 10 tham số đầu vào 
# => 10 giá trị đầu ra, mỗi đầu ra có 2 hàng xác suất để lấy kết quả

parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}
a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)

print("xt = ", xt, end='\n \n ')
print("xt.shape = ", xt.shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("a_next = ", a_next, end='\n \n ')
print("a_next.shape = ", a_next.shape, end='\n \n')

print("-------------------------------------------------------------------------- \n")

print("yt_pred =", yt_pred, end='\n \n')
print("yt_pred.shape = ", yt_pred.shape, end='\n \n')

xt =  [[-0.44712856  1.2245077   0.40349164  0.59357852 -1.09491185  0.16938243
   0.74055645 -0.9537006  -0.26621851  0.03261455]
 [-1.37311732  0.31515939  0.84616065 -0.85951594  0.35054598 -1.31228341
  -0.03869551 -1.61577235  1.12141771  0.40890054]
 [-0.02461696 -0.77516162  1.27375593  1.96710175 -1.85798186  1.23616403
   1.62765075  0.3380117  -1.19926803  0.86334532]]
 
 xt.shape =  (3, 10)
 
 -------------------------------------------------------------------------- 

a_next =  [[-0.44749928 -0.99856195 -0.99950181  0.99131634 -0.3090681   0.68206768
  -0.43190526  0.97389606  0.96411712  0.31440968]
 [-0.99902289 -0.60394564  0.77687886 -0.76769407 -0.99915322  0.04749149
   0.83147629 -0.89684166 -0.96872602  0.52059961]
 [-0.99302934  0.95368488  0.97616036 -0.97478035  0.28672263 -0.9878114
  -0.69060207 -0.65883977 -0.99018297 -0.89495424]
 [-0.8485068   0.69099567 -0.72056723 -0.94896553  0.11901586  0.82262522
  -0.18972433  0.13202991  0.08415572 -0.14544515]
 [ 0.9

<img src="images/RNN.png" style="width:500;height:300px;">

In [5]:
def rnn_forward(x, a0, parameters):
    """
    Thực hiện truyền thuận của RNN.

    Arguments:
        x -- Dữ liệu đầu vào cho mỗi bước thứ t, có dạng (n_x, m, T_x).
        a0 -- Trạng thái ẩn ban đầu, có dạng (n_a, m)

    parameters -- Dictionary chứa:
        Waa -- Ma trận trọng số nhân với trạng thái ẩn, mảng numpy có dạng (n_a, n_a)
        Wax -- Ma trận trọng số nhân với đầu vào, mảng numpy có dạng (n_a, n_x)
        Wya -- Ma trận trọng số liên quan đến trạng thái ẩn và đầu ra, mảng numpy có dạng (n_y, n_a)
        ba --  Sai số, mảng numpy có dạng (n_a, 1)
        by -- Sai số liên quan đến trạng thái ẩn và đầu ra, mảng numpy có dạng (n_y, 1)

    Returns:
        a -- Các trạng thái ẩn cho mỗi bước thứ t, mảng numpy có dạng (n_a, m, T_x)
        y_pred -- Dự đoán cho mỗi bước thứ t, mảng numpy có dạng (n_y, m, T_x)
        caches -- tuple các giá trị cần thiết cho quá trình lan truyền ngược, chứa (list các caches, x)
    """
   
    # Khởi tạo "caches" chứa danh sách tất cả các caches
    caches = []
   
    # Trích xuất các kích thước từ dạng của x và Wy
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
   
    # Khởi tạo "a" và "y" với các giá trị zero 
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
   
    # Khởi tạo a_next 
    a_next = a0
   
    # Lặp qua tất cả các bước thời gian
    for t in range(T_x):
        # Cập nhật trạng thái ẩn tiếp theo, tính toán dự đoán, lấy cache
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)

        # Lưu giá trị của trạng thái ẩn mới "next" vào a
        a[:,:,t] = a_next

        # Lưu giá trị của dự đoán vào y 
        y_pred[:,:,t] = yt_pred
       
        # Thêm "cache" vào "caches"
        caches.append(cache)
   
    # lưu các giá trị cần thiết cho lan truyền ngược trong cache
    caches = (caches, x)
   
    return a, y_pred, caches

In [6]:
np.random.seed(1)

x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)

Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)

ba = np.random.randn(5,1)
by = np.random.randn(2,1)

parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}
a, y_pred, caches = rnn_forward(x, a0, parameters)

print("x[:,:,3] =", x[:,:,3], end='\n \n ')
print("x[:,:,3].shape = ", x[:,:,3].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("y_pred[:,:,3] =", y_pred[:,:,3], end='\n \n ')
print("y_pred[:,:,3].shape = ", y_pred[:,:,3].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("a[:,:,3] = ", a[:,:,3], end='\n \n ')
print("a[:,:,3].shape = ", a[:,:,3].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("caches[0][3] =", caches[0][3], end='\n \n ')
print("len(caches[0][3]) = ", len(caches[0][3]), end='\n \n ')

x[:,:,3] = [[-1.07296862 -0.7612069  -2.06014071 -1.09989127  0.58281521  0.50249434
  -0.93576943 -0.39675353 -0.0126646   0.74204416]
 [ 1.6924546   2.10025514 -0.35224985  0.58662319  0.88514116 -0.29809284
   1.51981682 -0.50446586 -2.02220122  0.76201118]
 [ 0.41005165  0.37756379  0.18515642  0.07734007  0.69803203  0.59357852
  -0.9537006   0.31515939 -1.31228341  0.40890054]]
 
 x[:,:,3].shape =  (3, 10)
 
 -------------------------------------------------------------------------- 

y_pred[:,:,3] = [[0.91973    0.9882193  0.27662409 0.18484053 0.95645202 0.89913759
  0.33476872 0.64578294 0.34243125 0.99018762]
 [0.08027    0.0117807  0.72337591 0.81515947 0.04354798 0.10086241
  0.66523128 0.35421706 0.65756875 0.00981238]]
 
 y_pred[:,:,3].shape =  (2, 10)
 
 -------------------------------------------------------------------------- 

a[:,:,3] =  [[ 0.99998339  0.99999942  0.99999504  0.9999979   0.6007902   0.99902443
   0.999997    0.99792727  0.97797982  0.99994617]
 [ 0.4

## 2. RNN lan truyền ngược

<img src="images/rnn_cell_backprop.png" style="width:500;height:300px;"> <br>

In [7]:
def rnn_cell_backward(da_next, cache):
    """
    Thực hiện một bước lan truyền ngược

    Arguments:
        'da_next' -- Gradient của hàm mất mát theo trạng thái ẩn tiếp theo.
        'cache' -- Dictionary chứa các giá trị hữu ích (đầu ra của hàm 'rnn_cell_forward()').

    Returns: 'gradients' -- Dictionary chứa:
        'dx' -- Gradient của dữ liệu đầu vào, có hình dạng (n_x, m).
        'da_prev' -- Gradient của trạng thái ẩn trước đó, có hình dạng (n_a, m).
        'dWax' -- Gradient của trọng số từ đầu vào tới ẩn, có hình dạng (n_a, n_x).
        'dWaa' -- Gradient của trọng số từ ẩn tới ẩn, có hình dạng (n_a, n_a).
        'dba' -- Gradient của vector sai số, có hình dạng (n_a, 1).
    """
   
    # Trích xuất các giá trị từ cache
    (a_next, a_prev, xt, parameters) = cache
   
    # Trích xuất các giá trị từ parameters
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    ba = parameters["ba"]

    # tính gradient của tanh đối với a_next
    dtanh = (1 - a_next ** 2) * da_next

    # tính gradient của loss đối với Wax 
    dxt = np.dot(Wax.T, dtanh)
    dWax = np.dot(dtanh, xt.T)

    # tính gradient đối với Waa 
    da_prev = np.dot(Waa.T, dtanh)
    dWaa = np.dot(dtanh, a_prev.T)

    # tính gradient đối với ba
    dba = np.sum(dtanh, axis=1, keepdims=1)
   
    # Lưu các gradient vào dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}
   
    return gradients

In [8]:
np.random.seed(1)

xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)

Wax = np.random.randn(5,3)
Waa = np.random.randn(5,5)
Wya = np.random.randn(2,5)

ba = np.random.randn(5,1)
by = np.random.randn(2,1)

parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}
a_next, yt, cache = rnn_cell_forward(xt, a_prev, parameters)

da_next = np.random.randn(5,10)
gradients = rnn_cell_backward(da_next, cache)

print("gradients[\"dxt\"] =", gradients["dxt"], end='\n \n ')
print("gradients[\"dxt\"].shape =", gradients["dxt"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"da_prev\"] =", gradients["da_prev"], end='\n \n ')
print("gradients[\"da_prev\"].shape =", gradients["da_prev"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dWax\"] =", gradients["dWax"], end='\n \n ')
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dWaa\"] =", gradients["dWaa"], end='\n \n ')
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dba\"] =", gradients["dba"], end='\n \n ')
print("gradients[\"dba\"].shape =", gradients["dba"].shape, end='\n \n ')

gradients["dxt"] = [[ 0.09154157 -0.11111027 -0.46975064 -0.27025034 -0.7174677   0.0660919
  -0.0751071  -0.17887198  0.15182069 -0.42552337]
 [ 0.06438856 -0.11849998 -1.38721305 -0.47076946 -0.53773545 -0.18484839
  -0.16660121  0.09614987  0.29431165  0.28386219]
 [-0.02886535  0.15285269 -0.24771553 -0.1538868  -0.30123978  0.43068929
   0.1448896   0.07010807 -0.22645813  0.09275214]]
 
 gradients["dxt"].shape = (3, 10)
 
 -------------------------------------------------------------------------- 

gradients["da_prev"] = [[-4.36474348e-02 -2.26754831e-03  1.08276087e+00  5.06680372e-01
   9.09167382e-01 -1.01613713e+00 -3.82226015e-01  5.24810753e-02
   7.23491246e-01  6.73910291e-02]
 [ 1.86197024e-01 -1.09694965e-01 -7.93539324e-01 -5.84463044e-01
  -2.89490632e+00  1.05808183e+00  9.32074819e-02  2.03106934e-01
  -1.23486448e-01  4.63043021e-01]
 [ 5.88433033e-02  8.27525487e-02  2.64035408e-02 -1.52399494e-01
  -6.73119533e-01 -4.00240713e-01 -3.65810558e-01 -1.70742048e-01
 

In [9]:
def rnn_backward(da, caches):
    """
    Thực hiện quá trình lan truyền ngược

    Arguments:
        da -- Thông số Gradient từ các trạng thái, có hình dạng (n_a, m, T_x)
        caches -- Tuple chứa thông tin từ quá trình truyền thuận (rnn_forward)
   
    Returns: gradients -- Dictionary chứa:
        dx -- Gradient đối với dữ liệu đầu vào, numpy-array có hình dạng (n_x, m, T_x)
        da0 -- Gradient đối với trạng thái ẩn ban đầu, numpy-array có hình dạng (n_a, m)
        dWax -- Gradient đối với ma trận trọng số đầu vào, numpy-array có hình dạng (n_a, n_x)
        dWaa -- Gradient đối với ma trận trọng số từ trạng thái ẩn tới trạng thái ẩn, numpy-array có hình dạng (n_a, n_a)
        dba -- Gradient đối với vector sai số, có hình dạng (n_a, 1)
    """
   
    # Trích xuất các giá trị từ cache đầu tiên (t=1) của caches 
    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[0]
   
    # Trích xuất các kích thước từ hình dạng của da và x1
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
   
    # khởi tạo các gradient với kích thước phù hợp 
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
       
    # Lặp qua tất cả các bước thời gian
    for t in reversed(range(T_x)):
        # Tính toán gradient tại bước thời gian t. Chọn "da_next" và "cache" để sử dụng trong bước truyền ngược.
        gradients = rnn_cell_backward(da[:,:,t] + da_prevt, caches[t])

        # Trích xuất đạo hàm từ gradients
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dWaa"], gradients["dba"]
        
        # Tăng hoặc giảm tùy vào xử lý đạo hàm toàn cục đối với các tham số bằng cách thêm đạo hàm của chúng tại bước thời gian t
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
       
    # Đặt da0 bằng gradient của a đã được truy ngược qua tất cả các bước thời gian là giá trị ban đầu
    da0 = da_prevt

    # Lưu trữ các gradient trong một từ điển python
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
   
    return gradients

In [10]:
np.random.seed(1)

x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Wax = np.random.randn(5,3)
Waa = np.random.randn(5,5)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)

parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}
a, y, caches = rnn_forward(x, a0, parameters)

# Các giá trị gradient sẽ được dùng để tối ưu các trọng số nhằm giảm giá trị loss
# => Có thể sử dụng thuật toán "Descent Gradient", hoặc tùy vào cách tối ưu cho từng bài toán cụ thể

da = np.random.randn(5, 10, 4)
gradients = rnn_backward(da, caches)

print("gradients[\"dx\"][:,:,1] =", gradients["dx"][:,:,1], end='\n \n ')
print("gradients[\"dx\"].shape =", gradients["dx"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"da0\"] =", gradients["da0"], end='\n \n ')
print("gradients[\"da0\"].shape =", gradients["da0"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dWax\"] =", gradients["dWax"], end='\n \n ')
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dWaa\"] =", gradients["dWaa"], end='\n \n ')
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dba\"] =", gradients["dba"], end='\n \n ')
print("gradients[\"dba\"].shape =", gradients["dba"].shape, end='\n \n ')

gradients["dx"][:,:,1] = [[ 0.4129776  -0.43110689  1.23379776  0.12563397  0.8497807   0.0485883
  -2.39779203 -4.44663604 -0.51404067  0.03956839]
 [-0.21351686 -0.79843759 -0.59255627 -0.17143457 -0.44729129  0.05234244
  -4.58628626  6.68951697  0.16346652 -0.17645862]
 [ 0.19405302  0.46577814  0.77017293 -0.46933372  0.46972456  0.02785356
   0.438081   17.20230784 -0.46312164  0.12418912]]
 
 gradients["dx"].shape = (3, 10, 4)
 
 -------------------------------------------------------------------------- 

gradients["da0"] = [[ 0.52867174 -1.43267105  0.17053642  1.46206818  0.45058923  1.37835668
  -0.50487     2.49399971  0.05036774 -0.45402172]
 [-0.13305141  0.66876495 -0.4412652  -0.28388415  1.01611216 -2.4581769
   1.09357817 -0.70874682  0.07428086 -0.78971378]
 [ 0.13004217  0.39457912 -0.46922503 -0.31494238  1.19294791  1.31399546
   0.60545664 -0.10066964  0.06550149 -1.0215378 ]
 [ 0.46754558  0.68863752  0.55637735 -0.74940635  3.1682477  -5.20418125
   1.13008319 -

In [11]:
def clip(gradients, maxValue):
    for gradient in gradients:
        np.clip(gradients[gradient], -maxValue, maxValue, out=gradients[gradient])
    
    return gradients

In [12]:
gradients = clip(gradients, 5)

In [13]:
print("gradients[\"dx\"][:,:,1] =", gradients["dx"][:,:,1], end='\n \n ')
print("gradients[\"dx\"].shape =", gradients["dx"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"da0\"] =", gradients["da0"], end='\n \n ')
print("gradients[\"da0\"].shape =", gradients["da0"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dWax\"] =", gradients["dWax"], end='\n \n ')
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dWaa\"] =", gradients["dWaa"], end='\n \n ')
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape, end='\n \n ')

print("-------------------------------------------------------------------------- \n")

print("gradients[\"dba\"] =", gradients["dba"], end='\n \n ')
print("gradients[\"dba\"].shape =", gradients["dba"].shape, end='\n \n ')

gradients["dx"][:,:,1] = [[ 0.4129776  -0.43110689  1.23379776  0.12563397  0.8497807   0.0485883
  -2.39779203 -4.44663604 -0.51404067  0.03956839]
 [-0.21351686 -0.79843759 -0.59255627 -0.17143457 -0.44729129  0.05234244
  -4.58628626  5.          0.16346652 -0.17645862]
 [ 0.19405302  0.46577814  0.77017293 -0.46933372  0.46972456  0.02785356
   0.438081    5.         -0.46312164  0.12418912]]
 
 gradients["dx"].shape = (3, 10, 4)
 
 -------------------------------------------------------------------------- 

gradients["da0"] = [[ 0.52867174 -1.43267105  0.17053642  1.46206818  0.45058923  1.37835668
  -0.50487     2.49399971  0.05036774 -0.45402172]
 [-0.13305141  0.66876495 -0.4412652  -0.28388415  1.01611216 -2.4581769
   1.09357817 -0.70874682  0.07428086 -0.78971378]
 [ 0.13004217  0.39457912 -0.46922503 -0.31494238  1.19294791  1.31399546
   0.60545664 -0.10066964  0.06550149 -1.0215378 ]
 [ 0.46754558  0.68863752  0.55637735 -0.74940635  3.1682477  -5.
   1.13008319 -0.761857

In [14]:
def update_parameters(a0, gradients, parameters, lr=0.01):

    a0 += -lr * gradients['da0']
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['ba']  += -lr * gradients['dba']

    return a0, parameters

In [15]:
a0, parameters = update_parameters(a0, gradients, parameters)

In [16]:
print(a0)
print(parameters)

[[-0.02990367 -0.76083491  1.27205057  1.95248107 -1.86248776  1.22238046
   1.63269945  0.3130717  -1.19977171  0.86788553]
 [-0.17958979 -0.61060828 -1.22564548  0.55337634  0.78264574 -0.59894896
   0.50964056 -1.13725392  0.80111822  0.05446444]
 [-0.18787019 -0.10569166  0.87357841  0.75356106  0.51753585  0.12456126
   0.07176656  0.61938696  0.23183954  0.69276678]
 [-0.31479223 -2.44172414  1.03326083  2.19447371  0.40968197 -0.05015523
  -0.14774558 -0.11143562  0.01584746 -1.10357968]
 [-0.51650197 -1.0040196   0.24768764 -0.28364711  0.49986946 -0.20975586
   0.98722368  0.22571423  2.19127466 -1.89918504]]
{'Wax': array([[-0.66672931,  0.95148689,  2.53048071],
       [-0.25842644,  0.00669928, -0.22787322],
       [ 1.31282912, -0.33730786,  0.63006984],
       [-0.33138997, -1.32255876,  0.29634408],
       [ 0.48397998,  1.28758702, -0.11280299]]), 'Waa': array([[-0.66736206,  0.5127611 ,  0.27310267,  0.33066508, -0.1231127 ],
       [ 1.2056807 ,  0.36785907,  1.881625